# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, when, col, desc, udf, col, sort_array, asc, avg
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import pandas as pd

# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
# TODO: write your code to answer question 1
user_log.select("page").where(user_log.userId == "").dropDuplicates().show()

+-----+
| page|
+-----+
| Home|
|About|
|Login|
| Help|
+-----+



# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


In [32]:
user_log.where(user_log.userId == "").collect()

[Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=0, lastName=None, length=None, level='free', location=None, method='PUT', page='Login', registration=None, sessionId=5598, song=None, status=307, ts=1513721196284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=26, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, sessionId=428, song=None, status=200, ts=1513721274284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='free', location=None, method='GET', page='Home', registration=None, sessionId=2941, song=None, status=200, ts=1513722009284, userAgent=None, userId=''),
 Row(artist=None, auth='Logged Out', firstName=None, gender=None, itemInSession=5, lastName=None, length=None, level='paid', location=None, method='GET', page='Home', registration=None, 

In [33]:
# TODO: use this space to explore the behavior of the user with an empty string
df = pd.DataFrame(user_log.where(user_log.userId == "").collect())
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,None,Logged Out,None,None,0,None,None,free,None,PUT,Login,None,5598,None,307,1513721196284,None,
1,None,Logged Out,None,None,26,None,None,paid,None,GET,Home,None,428,None,200,1513721274284,None,
2,None,Logged Out,None,None,5,None,None,free,None,GET,Home,None,2941,None,200,1513722009284,None,
3,None,Logged Out,None,None,5,None,None,paid,None,GET,Home,None,5243,None,200,1513723183284,None,
4,None,Logged Out,None,None,6,None,None,paid,None,PUT,Login,None,5243,None,307,1513723184284,None,


# Question 3

How many female users do we have in the data set?

In [34]:
# TODO: write your code to answer question 3
user_log.where(user_log.gender == "F").select(["userId", "gender"]).dropDuplicates().count()

462

# Question 4

How many songs were played from the most played artist?

In [36]:
# TODO: write your code to answer question 4
user_log.filter(user_log.page == "NextSong") \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [38]:
# TODO: write your code to answer question 5
function = udf(lambda ishome : int(ishome == 'Home'), IntegerType())

user_window = Window \
    .partitionBy('userID') \
    .orderBy(desc('ts')) \
    .rangeBetween(Window.unboundedPreceding, 0)

cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
    .select('userID', 'page', 'ts') \
    .withColumn('homevisit', function(col('page'))) \
    .withColumn('period', Fsum('homevisit').over(user_window))

cusum.filter((cusum.page == 'NextSong')) \
    .groupBy('userID', 'period') \
    .agg({'period':'count'}) \
    .agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

